In [ ]:
# Import library

import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
# Data Pre Processing

data_dir = 'drive/MyDrive/khasna/rps-cv-images' # call data directory
classes = os.listdir(data_dir)

height = 128 # image height
width = 128 # image width

with open('image_label.txt', 'w') as f:
    for image_class in classes:
        f.write(f'{image_class}\n')

data = []
labels = []

for dirname, _, filenames in os.walk(data_dir):
    data_class = dirname.split(os.path.sep)[-1]
    for filename in filenames:
        img_path = os.path.join(dirname, filename)

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (height , width))

        data.append(image)
        labels.append(classes.index(data_class))

# Ubah data ke numpy array
data = np.array(data)
labels = np.array(labels)

labels = to_categorical(labels)

In [ ]:
# Splitting Data

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1, random_state=42)

In [ ]:
# Checking Data

num_training_images = len(X_train)
print("Total training images:", num_training_images)
num_test_images = len(X_test)
print("Total test images:", num_test_images)

Total training images: 1969
Total test images: 219


In [ ]:
# Image Augmentation Process

train_data = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    zoom_range = 0.2,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

train_data.fit(X_train)

test_data = ImageDataGenerator(rescale = 1./255)

test_data.fit(X_test)

In [ ]:
# Buat arsitektur model neural network (menggunakan CNN)

def create_model():
    model = tf.keras.models.Sequential()
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(width, height, 3)))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu'))
    model.add(Dense(len(classes), activation = 'softmax'))
    model.summary()
    return model

In [ ]:
# Summary model

image_model = create_model()
image_model

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 128, 128, 64)      1792      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 64, 64, 64)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 62, 62, 64)        36928     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 31, 31, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 29, 29, 64)        36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 14, 14, 64)      

In [ ]:
image_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
image_model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
62/62 [==============================] - 139s 2s/step - loss: 16.3951 - accuracy: 0.7349 - val_loss: 0.2847 - val_accuracy: 0.9132
Epoch 2/5
62/62 [==============================] - 122s 2s/step - loss: 0.1595 - accuracy: 0.9421 - val_loss: 0.1689 - val_accuracy: 0.9498
Epoch 3/5
62/62 [==============================] - 122s 2s/step - loss: 0.0607 - accuracy: 0.9807 - val_loss: 0.0835 - val_accuracy: 0.9726
Epoch 4/5
62/62 [==============================] - 121s 2s/step - loss: 0.0525 - accuracy: 0.9827 - val_loss: 0.1733 - val_accuracy: 0.9361
Epoch 5/5
62/62 [==============================] - 118s 2s/step - loss: 0.0266 - accuracy: 0.9898 - val_loss: 0.0975 - val_accuracy: 0.9680


In [ ]:
# Cek akurasi model

loss, accuracy = image_model.evaluate(X_test, y_test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

In [ ]:
# Simpan model

image_model.save('image_model')